In [1]:
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [2]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFile, CogniteFileApply
from IPython.display import display

In [3]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [4]:
SPACE = "sp_upgrade"

In [5]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Happy-Path

In [6]:
file = FileMetadataWrite(
    external_id="file_case1_happy_path",
    name="File Case 1 Happy Path",
    mime_type="text/plain",
)
cognite_file = CogniteFileApply(space=SPACE, external_id=file.external_id, name=file.name)

In [7]:
from cognite.client.data_classes.capabilities import FilesAcl

client.token.get_scope([FilesAcl.Action.Write])

[AllScope()]

In [8]:
# Ensure the file does not exist before running the test
try:
    client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
except CogniteException as _:
    try:
        client.data_modeling.instances.delete(cognite_file.as_id())
    except CogniteException as e:
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print("\033[92mFile node deleted successfully\033[0m")
else:
    print("\033[92mFile deleted successfully\033[0m")

File node deleted successfully


In [9]:
created, upload_link = client.files.create(file)
created

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
labels,[]
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:33.607000
uploaded,False


In [10]:
filecontent = b"This is filecontent"

In [11]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:33.607000
uploaded,False


In [12]:
retrieved = client.files.retrieve(external_id=created.external_id)

In [13]:
retrieved

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:33.607000
uploaded,True
uploaded_time,2025-07-28 11:12:35.087000


In [14]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    display(updated)

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:38.768000
uploaded,True
uploaded_time,2025-07-28 11:12:35.087000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."


In [15]:
retrieved2 = client.files.retrieve(external_id=created.external_id)
retrieved2

,value
external_id,file_case1_happy_path
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:38.768000
uploaded,True
uploaded_time,2025-07-28 11:12:35.087000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."


In [16]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,file_case1_happy_path,1,True,2025-07-28 11:12:48.766,2025-07-28 11:12:48.766


In [17]:
retrieved3 = client.files.retrieve(external_id=created.external_id)
retrieved3

,value
external_id,file_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:38.768000
uploaded,True
uploaded_time,2025-07-28 11:12:35.087000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."


In [18]:
retrieved_content = client.files.download_bytes(instance_id=cognite_file.as_id())

In [19]:
retrieved_content

b'This is filecontent'

In [20]:
retrieved_node = client.data_modeling.instances.retrieve_nodes(cognite_file.as_id(), node_cls=CogniteFile)

In [21]:
retrieved_node

,value
space,sp_upgrade
external_id,file_case1_happy_path
version,2
last_updated_time,2025-07-28 11:12:52.242000
created_time,2025-07-28 11:12:48.766000
instance_type,node
name,File Case 1 Happy Path
description,None
tags,None
aliases,None


In [22]:
newcontent = b"This is new filecontent"

In [23]:
client.files.upload_content_bytes(newcontent, instance_id=cognite_file.as_id())

,value
external_id,file_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:38.768000
uploaded,True
uploaded_time,2025-07-28 11:12:35.087000


In [24]:
retrieved_updatedcontent = client.files.download_bytes(external_id=file.external_id)

In [25]:
retrieved_updatedcontent

b'This is new filecontent'

In [26]:
retrieved_file = client.files.retrieve(external_id=file.external_id)
retrieved_file

,value
external_id,file_case1_happy_path
instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."
name,File Case 1 Happy Path
mime_type,text/plain
id,5225636113511691
created_time,2025-07-28 11:12:33.607000
last_updated_time,2025-07-28 11:12:38.768000
uploaded,True
uploaded_time,2025-07-28 11:13:02.301000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'file_c..."


In [27]:
retrieved_cognite_file = client.data_modeling.instances.retrieve_nodes(cognite_file.as_id(), node_cls=CogniteFile)

In [28]:
retrieved_cognite_file.dump()

{'space': 'sp_upgrade',
 'externalId': 'file_case1_happy_path',
 'version': 3,
 'lastUpdatedTime': 1753701182749,
 'createdTime': 1753701168766,
 'instanceType': 'node',
 'properties': {'cdf_cdm': {'CogniteFile/v1': {'name': 'File Case 1 Happy Path',
    'description': None,
    'tags': None,
    'aliases': None,
    'sourceId': None,
    'sourceContext': None,
    'source': None,
    'sourceCreatedTime': None,
    'sourceUpdatedTime': None,
    'sourceCreatedUser': None,
    'sourceUpdatedUser': None,
    'assets': None,
    'mimeType': None,
    'directory': None,
    'isUploaded': True,
    'uploadedTime': '2025-07-28T11:13:02.301+00:00',
    'category': None}}}}